In [48]:
# 1

import lib
import importlib
import math
import random
importlib.reload(lib)

simx = lambda : random.normalvariate(0, 1)
stats = lib.RecurrentStats()

while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > 0.1:
    stats.add(simx())



print(stats.n)
print(stats.get_mean)
print(stats.get_var)
print(math.sqrt(stats.get_var))




111
-0.0655166238857105
1.1023562439210208
1.049931542492662


In [98]:
# 2

import lib
import importlib
import math
import random
importlib.reload(lib)

# a)
print("a")
# i

"""
Como esta integral es la esperanza E[(exp(x) / sqrt(2 * x))] con x ~ U(0,1)
Podemos hacerla usando Montecarlo directamente
"""
gi = lambda x : (math.exp(x) / math.sqrt(2 * x))
x = lib.montecarlo_multi(gi, [(0, 1)], 10_000)
print(f"i) {x}")

# ii
"""
A esta la separamos en dos: (-inf, 0) + (0, inf)
Haciendo un cambio de variable sobre la parte negativa, nos quedan dos integrales:

integral 0 a inf de f(x) dx + integral 0 a inf de f(-x) dx

en el apunte vimos como aproximar funciones de 0 a infinito. Usamos esa técnica abajo
"""
gii = lambda x : x**2 * math.exp(-x**2)
y = lib.montecarlo_multi(gii, [(0, math.inf)], 100_000) + lib.montecarlo_multi(lambda x : gii(-x), [(0, math.inf)], 100_000)


print(f"ii) {y}")

print("b")

# uso el metodo de montecarlo con n = 1, lo que me da una sola sample en vez de un promedio
# abajo hago el promedio parando cuando llegamos a la cota
simi = lambda : lib.montecarlo_multi(gi, [(0, 1)], 1)
simii = lambda : lib.montecarlo_multi(gii, [(0, math.inf)], 1) + lib.montecarlo_multi(lambda x : gii(-x), [(0, math.inf)], 1)

stats = lib.RecurrentStats()
while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > 0.01:
    stats.add(simi())
print("i) ", stats.mean)

stats = lib.RecurrentStats()
while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > 0.01:
    stats.add(simii())
print("ii)", stats.mean)



a
i) 2.0795610174226904
ii) 0.8887385388118403
b
i)  2.073377895501431
ii) 0.893259762690186


In [108]:
# 3

import lib
import importlib
import math
import random
importlib.reload(lib)


gi  = lambda x : math.sin(x) / x

simi  = lambda : lib.montecarlo_multi(gi, [(math.pi, 2*math.pi)], 1)
stats = lib.RecurrentStats()
zalpha_2 = 1.96 # alpha = 0.05
L = 2 * 0.001
d = L / (2 * zalpha_2)
while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > d:
    stats.add(simi())
print(stats.get_mean)
print(stats.n)

gii = lambda x : 3 / (3 + x**4)
simii = lambda : lib.montecarlo_multi(gii, [(0, math.inf)], 1)
stats = lib.RecurrentStats()
while (stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > d):
    stats.add(simii())
print(stats.get_mean)
print(stats.n)

-0.43436454959897297
169649
1.4622045389862766
3659704


In [6]:
import lib
import importlib
import math
import random
importlib.reload(lib)

# 4

print("b")

def simN():
    n = 0
    s = 0
    while s <= 1:
        s += random.random()
        n += 1
    return n

# esp, var = lib.sim_esp_var(simN, 10_000)
# print(esp, var)


print("c")
stats = lib.RecurrentStats()
zalpha_2 = 1.96 # alpha = 0.05
L = 0.025
d = L / (2 * zalpha_2)
while stats.n <= 100 or math.sqrt(stats.get_var / stats.n) > d:
    stats.add(simN())
print(stats.get_mean)

b
c
2.726426928705825


In [1]:
import lib
import importlib
import math
import random
importlib.reload(lib)

# 5

def simM():
    n = 1
    u_1 = random.random()
    
    while True:
        u_2 = random.random()
        n += 1


        if u_1 > u_2:
            break
        u_1 = u_2
        u_2 = random.random()
    return n


print("c")
stats = lib.RecurrentStats()
d = 0.01
while stats.n <= 100 or (stats.get_var/stats.n) > d:
    stats.add(simM())
print(stats.get_mean)
print(stats.n)

print("d")
stats = lib.RecurrentStats()
zalpha_2 = 1.96 # alpha = 0.05
L = 0.1
d = L / (2 * zalpha_2)
while stats.n <= 100 or math.sqrt(stats.get_var/stats.n) > d:
    stats.add(simM())
print(stats.get_mean)
print(stats.n)

c
2.7425742574257432
101
d
2.7576419213973806
1374


In [143]:

import lib
import importlib
import math
import random
importlib.reload(lib)
# 6

def g():
    x = 2 * random.random() - 1
    y = 2 * random.random() - 1
    return x**2 + y**2 <= 1

stats = lib.RecurrentStats()

while stats.n <= 100 or (stats.get_var/stats.n) > 0.0001:
    stats.add(g())
print("a")
print(stats.get_mean)
print(stats.n)

print("b")
L = 0.1
zalpha_2 = 1.96
d = L/(2 * zalpha_2)
stats = lib.RecurrentStats()

while stats.n <= 100 or math.sqrt(stats.get_var/stats.n) > d:
    stats.add(g())
print(stats.get_mean)
print(stats.n)

a
0.8113577023498683
1532
b
0.7302631578947371
304


In [148]:
import numpy as np
import lib
import importlib
import math
import random
importlib.reload(lib)
# 7 

# Distribucion teorica de Mendel 
# blanco, rojo, rosa
# 0, 1, 2
ps = [1/4, 1/4, 1/2]

print("a")

# Calculamos el estadistico T
# e 141 produjeron flores blancas, 291 flores rosas y 132 flores rojas
Ns = [141, 132, 291]
# get_estadistico_prueba
T = ((N - len(Ns)*ps[i])**2 / len(Ns)*ps[i] for (i, N) in enumerate(Ns))




def p(n): 
    try:
        return ps[n]
    except:
        return 0

sim_mendel = lambda : lib.trans_inversa_general_optimizada(p, [0, 1, 2])

psim = np.zeros(3)

n = 10_000

for _ in range(n):
    psim[sim_mendel()] += 1

print(psim/n)




[0. 0. 0.]
[0.2461 0.2583 0.4956]
